# The Labor Gap in Technical Demand and the Impact of Gender


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">
            <span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span>
        </li>    
        <li><span><a href="#Data-Manipulation" data-toc-modified-id="Data-Manipulation-2">
            <span class="toc-item-num">2&nbsp;&nbsp;</span>Data Manipulation</a></span>
        </li>
        <li>
            <span><a href="#Tree-Maps" data-toc-modified-id="Tree-Maps-3">
                <span class="toc-item-num">3&nbsp;&nbsp;</span>Tree Maps</a></span>
        </li>
        <li>
            <span><a href="#Heat-Maps" data-toc-modified-id="Heat-Maps-4">
                <span class="toc-item-num">4&nbsp;&nbsp;</span>Heat Maps</a></span>
        </li>
        <li>
            <span><a href="#Time-Series-Analysis" data-toc-modified-id="Time-Series-Analysis-5">
                <span class="toc-item-num">5&nbsp;&nbsp;</span>Time Series Analysis</a></span>
        </li>                 
        <li>
            <span><a href="#Conclusion" data-toc-modified-id="Conclusion-6">
                <span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusion</a>
            </span>
        </li>          
    </ul>

In [23]:
%pip install -r requirements.txt # for intial use, please run this line to install the required packages

Note: you may need to restart the kernel to use updated packages.


In [8]:

# Import Dependencies
import sys
import os
import pandas as pd
import altair as alt
from squarify import normalize_sizes, squarify
from bokeh.plotting import figure
from bokeh.sampledata.sample_superstore import data
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from jupyter_bokeh.widgets import BokehModel # widgets third party extension will be required
from statsmodels.stats.proportion import proportions_ztest


# Add custom py file to directory to import functions
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Import custom data manipulation helper functions
from data_manipulation import get_short_names, get_format_parameters, get_df_list_final, get_bls_data_2002_to_2015, get_student_record_df


### !!!DOCKER CONTAINER FOR REPRODUCIBILITY

If you are running an anaconda environement, a python version other than 3.11 or Mac OS. You can you use Docker Desktop to build the environement.

Doker Files:
    - Dockerfile.yaml
    - docker-compose.yaml


Steps:
1. From the project directory, run the following command in your terminal: docker-compose up -d

2. Go to the log files in your docker container instance and search for the key word "copy".

3. Copy and paste the url provided by Jupyter Notebooks. Its should look LIKE this http://127.0.0.1:8888/tree?token=0397757e1569ed9c4b9a449e4c729de8d297c24c50a16a12.








# Introduction

## High Level Summary

Our project is intended to analyze the distribution of undergraduate majors at a state university compared against the national labor market, as reported by the U.S. Bureau of Labor Statistics (BLS). A recent study found that the number of bachelor’s degrees programs at 1,754 institutions across the U.S. respond dramatically to changes in labor market demand (Conzelmann, 2024). We are interested in whether this university is following with keeping up with labor market trends.

Additionally, we are seeking to uncover gender disparities in technical fields. According to the United States census bureau, women make up around half the national workforce, but are incredibly underrepresented in STEM fields (Martinez, 2021). Similarly, on a national scale women make up over half of bachelor’s degree graduates, but only 38.6 % of STEM graduates (Singh, 2020). Because of this, we are interested in exploring gender disparities in occupations with high supply of students but low labor market demand.

## Objectives

By analyzing university enrollment data and demand by major, we are aiming to explore the following questions:

Question 1: Is the university keeping up with the growing demand for professional workers (as opposed to vocational and entry-level workers), in respect to BLS projections?

Question 2: Is the university over or under indexing the BLS National Average?

Question 3: Do technical fields show gender-related disparities between students and workers? If there are, are these disparities trending downward?

# Data Manipulation

In [10]:

# Import list of level dataframes to build treemap visuals.
# Please refer to data_manipulation.py for data manipulation documentation.
# Per DRY principles, because the data_manipulation.py functions were required across all analysis, 
# data manipulation functions were centralized into a single py file 
# This was to support reusability, reproducibility and readability throughout the project.
# This insures each contributor works with the same dataset for reproducibility.

df_level_list = get_df_list_final()
df_level_list[0].head(5)

,year,l1,number_of_students_all_sum,number_of_students_men_sum,number_of_students_women_sum,number_of_students_unknown_sum,number_of_workers_all_sum,median_weekly_earnings_all_mean,number_of_workers_men_sum,median_weekly_earnings_men_mean,number_of_workers_women_sum,median_weekly_earnings_women_mean
0,2011,Architecture and engineering occupations,1133.0,844.0,289.0,0.0,2493,753.285714,2178,739.904762,315,0.000000
1,2011,"Arts, design, entertainment, sports, and media...",433.0,212.0,221.0,0.0,1464,446.368421,853,257.842105,613,136.578947
2,2011,Building and grounds cleaning and maintenance ...,0.0,0.0,0.0,0.0,3339,555.166667,2212,591.166667,1129,216.333333
3,2011,Business and financial operations occupations,2.0,1.0,1.0,0.0,5170,891.035714,2223,609.142857,2944,468.714286
4,2011,Community and social service occupations,0.0,0.0,0.0,0.0,1932,667.375000,729,342.125000,1202,386.500000


# Tree Maps

## Tree Map Data Manipulation

In [26]:

def treemap_data_manipulation(target_metric):
    # import BLS hierarchy mapping to build treemap block levels.
    df_occupation_level_mapping = pd.read_excel('./data/bls_cpsaat39_2011_to_2015.xlsx', sheet_name='level_mapping_l0', header=0)
    df_occupation_level_mapping_distinct = df_occupation_level_mapping[['l4', 'l3', 'l2', 'l1']].drop_duplicates().reset_index()
    df_occupation_level_mapping_distinct = df_occupation_level_mapping_distinct[['l4', 'l3', 'l2', 'l1']]
    df_occupation_level_mapping_distinct

    # Define the target metric to build treemap.
    metric = target_metric # metric = 'number_of_workers_all_sum'
    # Extract base metric exclusive of sex to be used by the get_short_names(level, metric) formatting function.
    base_metric = metric.replace('_sum', '').replace('_women', '').replace('_men', '').replace('_all', '')

    # Create level 1 and level 2 dataframes for treemap block data objects.
    l1_grouping = df_level_list[0]
    l2_grouping = df_level_list[1]

    # Merge hiearchny mapping to level 1 block grouping.
    l1_grouping = pd.merge(l1_grouping,
                            df_occupation_level_mapping_distinct[['l1', 'l2']], 
                            how='left',
                            left_on=['l1'],
                            right_on=['l1'])


    # Filter out the most recent year for analsyis by target metric.
    l1_grouping = l1_grouping[l1_grouping['year'] == '2015']
    l1_grouping = l1_grouping[['l1','l2', metric]]
    l2_grouping = l2_grouping[l2_grouping['year'] == '2015']
    l2_grouping = l2_grouping[['l2', metric]]

    # Get short name dictionary mapper
    short_name_l2 = get_short_names('l2', base_metric)
    # Transform BLS long name to short name for readability in treemap visual.
    l1_grouping['l2'] = l1_grouping['l2'].apply(lambda x: short_name_l2[x] if x in short_name_l2.keys() else x)
    l2_grouping['l2'] = l2_grouping['l2'].apply(lambda x: short_name_l2[x] if x in short_name_l2.keys() else x)
    # Transform BLS long name to short name for readability in treemap visual.
    short_name_l1 = get_short_names('l1', base_metric)
    l1_grouping['l1'] = l1_grouping['l1'].apply(lambda x: short_name_l1[x] if x in short_name_l1.keys() else x)

    # Compute total metric value to calculate worker percentages by level category.
    total = l2_grouping[metric].sum() # total seems low, validate later on

    # Concat percentage of workers to level labels for visual.
    l1_grouping['l1'] = l1_grouping.apply(lambda x: x['l1'] + ' | ' + str(int(round(x[metric]/total*100, 0))) + '%', axis=1)
    l2_grouping['l2'] = l2_grouping.apply(lambda x: x['l2'] + ' | ' + str(int(round(x[metric]/total*100, 0))) + '%', axis=1)
    l2_lookup = {k: k + " | " + v for k, v in (x.split(" | ") for x in l2_grouping['l2'].to_list())}
    l1_grouping['l2'] = l1_grouping.apply(lambda x: l2_lookup[x['l2']], axis=1)

    # keep only records with non-zero values, otherwise treemap will throw a divide by zero error
    l1_grouping = l1_grouping[l1_grouping[metric] > 0]
    l2_grouping = l2_grouping[l2_grouping[metric] > 0]

    return l1_grouping, l2_grouping


In [27]:

def build_project_treemap(target_metric='number_of_students_all_sum'):

    # Define the target metric to build treemap.
    metric = target_metric # metric = 'number_of_workers_all_sum'
    # Extract base metric exclusive of sex to be used by the get_short_names(level, metric) formatting function.
    base_metric = metric.replace('_sum', '').replace('_women', '').replace('_men', '').replace('_all', '')

    l1_grouping, l2_grouping = treemap_data_manipulation(target_metric)

    # The following code derived from: treemaps https://docs.bokeh.org/en/latest/docs/examples/topics/hierarchical/treemap.html
    # Fist function builds the tree blocks required to build the treemap visual. ie. parent level 2 blocks and child level 1 blocks.
    def treemap(df, col, x, y, dx, dy, *, N=100):
        sub_df = df.nlargest(N, col)
        normed = normalize_sizes(sub_df[col], dx, dy)
        blocks = squarify(normed, x, y, dx, dy)
        blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
        return sub_df.join(blocks_df, how='left').reset_index()

    # Get unique level 2 labels
    l2s = tuple(l2_grouping['l2'].unique().tolist())

    # define block 2 parameters 
    x, y, w, h = 0, 0, 2000, 1125
    blocks_by_L2 = treemap(l2_grouping, metric, x, y, w, h)

    blocks_by_L2.drop(columns=['index'], inplace=True)

    # define block 2 parameters select target columns need to supply def treemap function.
    blocks_by_L2 = blocks_by_L2[['l2', metric, 'x', 'y', 'dx', 'dy']] 

    # Iterate through each level 2 and build level 1 treemap blocks.
    dfs = []
    for index, (l2, number_of_workers_all, x, y, dx, dy) in blocks_by_L2.iterrows():
        df = l1_grouping[l1_grouping.l2==l2]    
        # Create level 1 treemap blocks and append to added to parent block 2
        dfs.append(treemap(df, metric, x, y, dx, dy, N=100))

    # Concat level 1 blocks
    blocks = pd.concat(dfs)

    # Create Bokeh figure object
    p = figure(width=w, height=h, tooltips="@l1", toolbar_location=None,
            x_axis_location=None, y_axis_location=None)
    p.x_range.range_padding = p.y_range.range_padding = 0
    p.grid.grid_line_color = None


    # Get treemap format parameters
    param_set = get_format_parameters(metric=base_metric, number_of_groups=len(l2_grouping))
    block_params = param_set['block']
    l2_params = param_set['l2']
    l1_params = param_set['l1']

    # Build block 2 ie. parent treemap graphical object 
    p.block('x', 'y', 'dx', 'dy', source=blocks, line_width=block_params['line_width'], line_color=block_params['line_color'],
            fill_alpha=block_params['fill_alpha'],
            fill_color=factor_cmap("l2", block_params['palette'][::-1] if 'women' in metric or 'all' in metric else block_params['palette'], l2s)) # legend_field="l2",  

    # Apply formates from get_format_parameters() helper function block 2 parents
    p.text('x', 'y',  x_offset=l2_params['x_offset'], y_offset=l2_params['y_offset'], text="l2", source=blocks_by_L2,
        text_font_size=l2_params['text_font_size'], text_color=l2_params['text_color'])

    blocks["ytop"] = blocks.y + blocks.dy

    # Apply formates from get_format_parameters() helper function to block 1 children
    p.text('x', 'ytop', x_offset=l1_params['x_offset'], y_offset=l1_params['y_offset'], text="l1", source=blocks,
        text_font_size=l1_params['text_font_size'], text_baseline=l1_params['text_baseline'],
        text_color=l1_params['text_color'])

    # function required to display bokeh plot in jupyter notebook
    output_notebook()
    # display() and BokehModel() required to display visual in jupyter notebook native VSCode IDE
    handle = display(BokehModel(p))


In [28]:
build_project_treemap(target_metric='number_of_workers_all_sum')

Loading BokehJS ...

BokehModel(render_bundle={'docs_json': {'9fae8085-2581-440c-b280-b258a541dc1b': {'version': '3.5.2', 'title': …

In [29]:
build_project_treemap(target_metric='number_of_students_all_sum')

Loading BokehJS ...

BokehModel(render_bundle={'docs_json': {'d4c0c657-7a0e-45da-8eaf-985e9a103d5e': {'version': '3.5.2', 'title': …

# Heat Maps

## Heat Map Data Manipulation

In [24]:
#Create Base Dataframe with Short Names for Heat Maps

import sys
sys.path.append('/data/data_manipulation.py')
from data_manipulation import get_df_list_final, get_short_names
 
df = get_df_list_final()[0]
df['l1_short_name'] = df.apply(lambda x: get_short_names('l1', 'default')[x['l1']], axis=1)

df.head()

,year,l1,number_of_students_all_sum,number_of_students_men_sum,number_of_students_women_sum,number_of_students_unknown_sum,number_of_workers_all_sum,median_weekly_earnings_all_mean,number_of_workers_men_sum,median_weekly_earnings_men_mean,number_of_workers_women_sum,median_weekly_earnings_women_mean,l1_short_name
0,2011,Architecture and engineering occupations,1133.0,844.0,289.0,0.0,2493,753.285714,2178,739.904762,315,0.000000,Arch & Eng
1,2011,"Arts, design, entertainment, sports, and media...",433.0,212.0,221.0,0.0,1464,446.368421,853,257.842105,613,136.578947,Ent
2,2011,Building and grounds cleaning and maintenance ...,0.0,0.0,0.0,0.0,3339,555.166667,2212,591.166667,1129,216.333333,Maintenance
3,2011,Business and financial operations occupations,2.0,1.0,1.0,0.0,5170,891.035714,2223,609.142857,2944,468.714286,Financial
4,2011,Community and social service occupations,0.0,0.0,0.0,0.0,1932,667.375000,729,342.125000,1202,386.500000,Social Services


In [65]:
def heatmap_final():

    # Build dataframe for heatmaps off of base df
    heatmap_final_women_df=df.copy()
    heatmap_final_men_df=df.copy()
    
    # Create restructured dataframe with 'Sex' as a feature, first for women
    heatmap_final_women_df['Sex']=['Women']*len(heatmap_final_women_df)
    heatmap_final_women_df['Students in Related Major']=df['number_of_students_women_sum']
    heatmap_final_women_df['Workers in Occupation']=df['number_of_workers_women_sum']
    
    heatmap_final_men_df['Sex']=['Men']*len(heatmap_final_men_df)
    heatmap_final_men_df['Students in Related Major']=df['number_of_students_men_sum']
    heatmap_final_men_df['Workers in Occupation']=df['number_of_workers_men_sum']
    
    # Combine the separate women and men dataframes vertically and sort
    heatmap_final_df = pd.concat([heatmap_final_women_df, heatmap_final_men_df],    
                              ignore_index = True,
                              sort = False)
    heatmap_final_df = heatmap_final_df.sort_values(by=['l1','year','Sex'],ascending=[True, True, False])
    
    # Calculate number_of_workers_total and numberof_students_total for each year and add these two features to heatmap_df
    totals_df = df.groupby(['year'])[['number_of_students_all_sum','number_of_workers_all_sum']].sum().reset_index()
    totals_df = totals_df.rename(columns={'year':'Year','number_of_students_all_sum': 'Number_of_Students_Total_Year', 
                                          'number_of_workers_all_sum': 'Number_of_Workers_Total_Year'})
    heatmap_final_df = pd.merge(heatmap_final_df, totals_df, how='left', left_on='year',right_on='Year')
    
    # Calculate workers and students proportions for each occupation and sex combination and add these two features to heatmap_df
    heatmap_final_df['Students%']=heatmap_final_df['Students in Related Major']/heatmap_final_df['Number_of_Students_Total_Year']
    heatmap_final_df['Workers%']=heatmap_final_df['Workers in Occupation']/heatmap_final_df['Number_of_Workers_Total_Year']
    
    # Reorder features and drop unnecessary features
    heatmap_final_df = heatmap_final_df[['Year','l1_short_name','Sex','Workers in Occupation','Number_of_Workers_Total_Year','Workers%',
                                         'Students in Related Major', 'Number_of_Students_Total_Year','Students%']]
    
    # Create 'Workers% - Students%' feature 
    heatmap_final_df['Workers% - Students%']=heatmap_final_df['Workers%']-heatmap_final_df['Students%']
    
    # Create 'Sex% Delta' features
    heatmap_final_df['Sex% Delta (Students)'] = heatmap_final_df.groupby(['Year','l1_short_name'])['Students%'].diff()
    heatmap_final_df['Sex% Delta (Students)']=[-heatmap_final_df['Sex% Delta (Students)'][i+1] if i%2 == 0 else
                                           heatmap_final_df['Sex% Delta (Students)'][i] for i in 
                                               range(len(heatmap_final_df['Sex% Delta (Students)']))]
    
    heatmap_final_df['Sex% Delta (Workers)'] = heatmap_final_df.groupby(['Year','l1_short_name'])['Workers%'].diff()
    heatmap_final_df['Sex% Delta (Workers)']=[-heatmap_final_df['Sex% Delta (Workers)'][i+1] if i%2 == 0 else
                                           heatmap_final_df['Sex% Delta (Workers)'][i] for i in 
                                              range(len(heatmap_final_df['Sex% Delta (Workers)']))]
        
    return heatmap_final_df


In [66]:
heatmap_final().head()

,Year,l1_short_name,Sex,Workers in Occupation,Number_of_Workers_Total_Year,Workers%,Students in Related Major,Number_of_Students_Total_Year,Students%,Workers% - Students%,Sex% Delta (Students),Sex% Delta (Workers)
0,2011,Arch & Eng,Women,315,100463,0.003135,289.0,6239.0,0.046322,-0.043186,-0.088957,-0.018544
1,2011,Arch & Eng,Men,2178,100463,0.021680,844.0,6239.0,0.135278,-0.113598,0.088957,0.018544
2,2012,Arch & Eng,Women,318,102754,0.003095,288.0,6159.0,0.046761,-0.043666,-0.100666,-0.018987
3,2012,Arch & Eng,Men,2269,102754,0.022082,908.0,6159.0,0.147427,-0.125345,0.100666,0.018987
4,2013,Arch & Eng,Women,332,104255,0.003184,306.0,6456.0,0.047398,-0.044213,-0.085812,-0.017985


In [80]:
def rev_heatmap_final():

    # Create revised dataframe for heat maps based on dataframe df
    rev_heatmap_workers_df=df.copy()
    rev_heatmap_students_df=df.copy()
    
    # Create restructured dataframe with 'Worker vs Student' classification as a feature, first for women
    rev_heatmap_workers_df['Employment Classification']=['Workers']*len(rev_heatmap_workers_df)
    rev_heatmap_workers_df['% Women in Sub-Population']=rev_heatmap_workers_df['number_of_workers_women_sum']/rev_heatmap_workers_df['number_of_workers_all_sum']
    
    rev_heatmap_students_df['Employment Classification']=['Students']*len(rev_heatmap_students_df)
    rev_heatmap_students_df['% Women in Sub-Population']=rev_heatmap_students_df['number_of_students_women_sum']/rev_heatmap_workers_df['number_of_students_all_sum']
    
    
    # Combine the separate workers and students dataframes vertically and sort
    rev_heatmap_df = pd.concat([rev_heatmap_workers_df, rev_heatmap_students_df],    
                              ignore_index = True,
                              sort = False)
    
    rev_heatmap_df['Women Workers %'] = rev_heatmap_df['number_of_workers_women_sum']/rev_heatmap_df['number_of_workers_all_sum']
    rev_heatmap_df['Women Students %'] = rev_heatmap_df['number_of_students_women_sum']/rev_heatmap_df['number_of_students_all_sum']
    rev_heatmap_df = rev_heatmap_df.sort_values(by=['l1','year','Employment Classification'],ascending=[True, True, False])
    rev_heatmap_df = rev_heatmap_df[rev_heatmap_df['number_of_students_all_sum']!=0]

    return rev_heatmap_df


In [81]:
rev_heatmap_final().head()

,year,l1,number_of_students_all_sum,number_of_students_men_sum,number_of_students_women_sum,number_of_students_unknown_sum,number_of_workers_all_sum,median_weekly_earnings_all_mean,number_of_workers_men_sum,median_weekly_earnings_men_mean,number_of_workers_women_sum,median_weekly_earnings_women_mean,l1_short_name,Employment Classification,% Women in Sub-Population,Women Workers %,Women Students %
0,2011,Architecture and engineering occupations,1133.0,844.0,289.0,0.0,2493,753.285714,2178,739.904762,315,0.00000,Arch & Eng,Workers,0.126354,0.126354,0.255075
110,2011,Architecture and engineering occupations,1133.0,844.0,289.0,0.0,2493,753.285714,2178,739.904762,315,0.00000,Arch & Eng,Students,0.255075,0.126354,0.255075
22,2012,Architecture and engineering occupations,1196.0,908.0,288.0,0.0,2589,764.095238,2269,782.761905,318,0.00000,Arch & Eng,Workers,0.122827,0.122827,0.240803
132,2012,Architecture and engineering occupations,1196.0,908.0,288.0,0.0,2589,764.095238,2269,782.761905,318,0.00000,Arch & Eng,Students,0.240803,0.122827,0.240803
44,2013,Architecture and engineering occupations,1166.0,860.0,306.0,0.0,2539,739.095238,2207,678.333333,332,44.47619,Arch & Eng,Workers,0.130760,0.130760,0.262436


## Heat Map Visuals

In [105]:
def heat_map(feature):
    
    heatmap_final_df = heatmap_final().copy()
    
    heatmap = alt.Chart(heatmap_final_df).mark_bar(size=20).encode(
            x=alt.X('Year:O'),
            y=alt.Y('l1_short_name:N', axis=alt.Axis(tickCount=5, title='')),
            color=alt.Color(feature,scale=alt.Scale(scheme='cividis'))
            ).properties(width=300, height=600
            ).facet(column=alt.Column('Sex:N',title="")).configure_view(strokeWidth=0
            )
        
    return heatmap

In [115]:
feature = 'Workers in Occupation'
heat_map(feature)

alt.FacetChart(...)

In [120]:
def rev_heat_map(feature):

    rev_heatmap_df = rev_heatmap_final().copy()

    string = '% Women in Sub-Population:Q'
    
    rev_heatmap = alt.Chart(rev_heatmap_df).mark_bar(size=20).encode(
            x=alt.X('year:O'),
            y=alt.Y('l1_short_name:N', axis=alt.Axis(tickCount=5, title='')),
            color=alt.Color(feature,scale=alt.Scale(scheme='cividis'))
            ).properties(width=200, height=400
            ).facet(column=alt.Column('Employment Classification:N',title="")).configure_view(strokeWidth=0
            )
    
    
    return rev_heatmap

In [121]:
# Use this function to create heat map Style 2
# Available features: % Women in Sub-Population

feature = '% Women in Sub-Population:Q'
rev_heat_map(feature)

alt.FacetChart(...)

#  Student and Worker Proportions by Field

## Pivot Table

In [30]:
# @AUSTIN!!!!! this excel sheet is the one you sent me. where did you get it/how did you make it?
data_to_be_pivoted = pd.read_excel("./data/df_merge_bls_l1_pivoted.xlsx", 
                                     sheet_name="df_merge_bls_l1", 
                                     header=0)
data_to_be_pivoted = data_to_be_pivoted[data_to_be_pivoted['year'] == 2015] 
data_to_be_pivoted.columns

# save total counts as variables
worker_total = data_to_be_pivoted['number_of_workers_all_sum'].sum()
student_total = data_to_be_pivoted['number_of_students_all_sum'].sum()

# create pivot table grouping the sum of students and workers for each occupational grouping
delta_table = pd.pivot_table(data_to_be_pivoted, values=['number_of_students_all_sum', 'number_of_workers_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)
delta_table = delta_table[delta_table['number_of_students_all_sum'] != 0]

# calculate the % of students in each field and the % of workers in each field
delta_table['% students'] = delta_table['number_of_students_all_sum']/student_total
delta_table['% workers'] = delta_table['number_of_workers_all_sum']/worker_total

# Calculate the difference between the % of students in each field and the % of workers in each field
delta_table['delta'] = delta_table['% workers'] - delta_table['% students']

#display entire table 
display(delta_table)

#display occupational feilds with large supply/demand differences 
delta_table[(delta_table['delta'] >= 0.06) | (delta_table['delta'] <= -0.06)]

,number_of_students_all_sum,number_of_workers_all_sum,% students,% workers,delta
l1,,,,,
Architecture and engineering occupations,761.0,2655.0,0.150514,0.024339,-0.126175
"Arts, design, entertainment, sports, and media occupations",351.0,1643.0,0.069422,0.015062,-0.054361
Business and financial operations occupations,40.0,5943.0,0.007911,0.054481,0.046570
Computer and mathematical occupations,422.0,4151.0,0.083465,0.038054,-0.045412
"Education, training, and library occupations",608.0,6886.0,0.120253,0.063126,-0.057127
Healthcare practitioners and technical occupations,8.0,6568.0,0.001582,0.060211,0.058629
Healthcare support occupations,118.0,2394.0,0.023339,0.021947,-0.001392
"Life, physical, and social science occupations",2330.0,1174.0,0.460839,0.010762,-0.450076
Management occupations,58.0,12479.0,0.011472,0.114399,0.102928


,number_of_students_all_sum,number_of_workers_all_sum,% students,% workers,delta
l1,,,,,
Architecture and engineering occupations,761.0,2655.0,0.150514,0.024339,-0.126175
"Life, physical, and social science occupations",2330.0,1174.0,0.460839,0.010762,-0.450076
Management occupations,58.0,12479.0,0.011472,0.114399,0.102928


## Z-test

In [31]:
# Calculate percentage of female students for each occupation where the absolute value of delta > 0.06
student_table = pd.pivot_table(data_to_be_pivoted[(data_to_be_pivoted['l1'] == "Architecture and engineering occupations") |(data_to_be_pivoted['l1'] == "Life, physical, and social science occupations") |(data_to_be_pivoted['l1'] == "Management occupations")],
                                values=['number_of_students_women_sum', 'number_of_students_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)
student_table['%women'] = ((student_table['number_of_students_women_sum']/student_table['number_of_students_all_sum']) * 100).round(decimals=0)
student_table.reset_index(inplace = True)
display(student_table)

# Calculate percentage of female workers for each occupation where the absolute value of delta > 0.06
workers_table = pd.pivot_table(data_to_be_pivoted[(data_to_be_pivoted['l1'] == "Architecture and engineering occupations") |(data_to_be_pivoted['l1'] == "Life, physical, and social science occupations") |(data_to_be_pivoted['l1'] == "Management occupations")],
                                values=['number_of_workers_women_sum', 'number_of_workers_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)
workers_table['%women'] = ((workers_table['number_of_workers_women_sum']/workers_table['number_of_workers_all_sum']) * 100).round(decimals=0)
workers_table.reset_index(inplace = True)
display(workers_table)

# Comparing the proportion of women students to women workers with Z tests for each field 
for l in student_table['l1']:
    X1 = student_table['number_of_students_women_sum'][student_table['l1'] == l].values[0]
    X2 = workers_table['number_of_workers_women_sum'][workers_table['l1'] == l].values[0]
    n1 = student_table['number_of_students_all_sum'][student_table['l1'] == l].values[0]
    n2 = workers_table['number_of_workers_all_sum'][workers_table['l1'] == l].values[0]

    # Number of successes in both groups
    count = [X1, X2]
    # Sample sizes for both groups
    nobs = [n1, n2]

    # Perform the two-proportion z-test
    stat, p_value = proportions_ztest(count, nobs)

    # Round the Z-statistic to 2 decimal places
    stat_rounded = round(stat, 2)
    # Format p_value to 3 decimal places, or print '< .001' if it's less than 0.001
    if p_value < 0.001:
        p_value_formatted = "< .001"
    else:
        p_value_formatted = round(p_value, 3)

    # print statements explaining differences between grousp
    if p_value < 0.05:
        print("The proportion of women students in {} fields are significantly higher than women workers in {} fields (Z = {}, p = {})"
              .format(l, l, stat_rounded, p_value_formatted))
    else:
        print("The proportion of women students in {} fields are NOT significantly different than women workers in {} fields (Z = {}, p = {})"
              .format(l, l, stat_rounded, p_value_formatted))


,l1,number_of_students_all_sum,number_of_students_women_sum,%women
0,Architecture and engineering occupations,761.0,211.0,28.0
1,"Life, physical, and social science occupations",2330.0,1366.0,59.0
2,Management occupations,58.0,31.0,53.0


,l1,number_of_workers_all_sum,number_of_workers_women_sum,%women
0,Architecture and engineering occupations,2655.0,382.0,14.0
1,"Life, physical, and social science occupations",1174.0,516.0,44.0
2,Management occupations,12479.0,5146.0,41.0


The proportion of women students in Architecture and engineering occupations fields are significantly higher than women workers in Architecture and engineering occupations fields (Z = 8.56, p = < .001)
The proportion of women students in Life, physical, and social science occupations fields are significantly higher than women workers in Life, physical, and social science occupations fields (Z = 8.22, p = < .001)
The proportion of women students in Management occupations fields are NOT significantly different than women workers in Management occupations fields (Z = 1.88, p = 0.06)


# Time Series Analysis

## Time Series Data Manipulation

In [32]:

merge_record_df = get_student_record_df()
df_bls_all_filtered = get_bls_data_2002_to_2015()

def data_prep(data1, data2):
    # Student Data Prep
    major_df = pd.read_excel("./data/majors.xlsx")
    student_Occupations_df = pd.merge(
                    data1,
                    major_df, 
                    how='left',
                    left_on='MAJOR1_DESCR',
                    right_on='MAJOR')

    pivot_table = pd.pivot_table(
        student_Occupations_df,
        index=['OCCUPATION', 'MAJOR1_TERM_TERM_YEAR'],  # Rows
        columns='SEX',                                  # Columns
        aggfunc='size',                                 # Count of occurrences
        fill_value=0                                    # Fill missing values with 0
    )
    pivot_table.reset_index(inplace=True)
    pivot_table['MAJOR1_TERM_TERM_YEAR'] = pivot_table['MAJOR1_TERM_TERM_YEAR'].apply(lambda x: int(x))

    student_melted = pd.melt(pivot_table, 
                        id_vars=['MAJOR1_TERM_TERM_YEAR', 'OCCUPATION'],  # Columns to keep
                        value_vars=['F', 'M'],  # Columns to unpivot
                        var_name='SEX',  # Name for new column indicating Sex (Females/Males)
                        value_name='count')  # Name for new column holding the values

    # BLS Data Prep
    def getMales(row):
        x = row['male_16_years_and_over'] + row['male_20_years_and_over']
        return x if isinstance(x, float) else None

    data2['Males'] = data2.apply(lambda x: getMales(x), axis=1)

    def getFemales(row):
        x = row['female_16_years_and_over'] + row['female_20_years_and_over']
        return x if isinstance(x, float) else None

    data2['Females'] = data2.apply(lambda x: getFemales(x), axis=1)

    table = pd.pivot_table(data2, values=['Males', 'Females'], index=['year', 'occupation'],
                           aggfunc="sum")
    table.reset_index(inplace=True)
    table = table[table['occupation'] != 'Unknown']
    table['year'] = table['year'].apply(lambda x: int(x))

    # Fix the filter here, use `table['year']` not `pivot_table['year']`

    # Melt the BLS data
    bls_melted = pd.melt(table, 
                        id_vars=['year', 'occupation'],  # Columns to keep
                        value_vars=['Females', 'Males'],  # Columns to unpivot
                        var_name='SEX',  # Name for new column indicating Sex (Females/Males)
                        value_name='count')  # Name for new column holding the values
    
    student_melted = student_melted.rename(columns={'MAJOR1_TERM_TERM_YEAR': 'year'})
    bls_melted = bls_melted.rename(columns={'occupation': 'OCCUPATION'})

    return student_melted, bls_melted

# Call the function
student_melted, bls_melted = data_prep(merge_record_df, df_bls_all_filtered)
student_melted.head(5)


,year,OCCUPATION,SEX,count
0,1982,Architecture and engineering occupations,F,0
1,1983,Architecture and engineering occupations,F,0
2,1984,Architecture and engineering occupations,F,2
3,1985,Architecture and engineering occupations,F,0
4,1986,Architecture and engineering occupations,F,2


## Time Series Visuals

In [33]:
def TimeSeriesBar(data, occupation, people):
    data = data.copy()
    data = data[(data['year'] > 2008) & (data['year'] <= 2016)]

    # Ensure the 'year' column is in the correct format (datetime)
    data['year'] = pd.to_datetime(data['year'].astype(str))
    data = data[data['OCCUPATION'] == occupation]

    # Define your custom color scheme
    custom_colors = [
        '#FBEA65',  # for Female
        '#081F49',  # for Male
    ]

    # Create the stacked bar chart with specified color order and no legend
    bar_chart = alt.Chart(data).mark_bar(size=15, stroke='black', strokeWidth=1).encode(
        x=alt.X("year:T",
                axis=alt.Axis(grid=False, title="Year"),
                scale=alt.Scale(domain=[pd.to_datetime('2008-01-01'), pd.to_datetime('2016-01-01')])
        ),
        y=alt.Y('count', stack=True, axis=alt.Axis(grid=False, title=people)),
        color=alt.Color("SEX", 
                        scale=alt.Scale(range=custom_colors),  # Use the custom colors for SEX
                        legend=None  # Remove the legend
        )
    ).properties(
        title={
            "text": f"{occupation}",
            "subtitle": f"Distribution by Sex for {people}",
            "anchor": "start"        # Align title to the left
        },
        width=200,    # Set custom width
        height=200   # Set custom height
    )
    
    return bar_chart


In [34]:
TimeSeriesBar(student_melted, 'Life, physical, and social science occupations', "Students")

alt.Chart(...)

In [35]:
TimeSeriesBar(bls_melted, 'Life, physical, and social science occupations', "Workers (in thousands)")

alt.Chart(...)

In [36]:
TimeSeriesBar(student_melted, 'Architecture and engineering occupations', "Students")

alt.Chart(...)

In [37]:
TimeSeriesBar(bls_melted, 'Architecture and engineering occupations', "Workers (in thousands)")

alt.Chart(...)

In [38]:
TimeSeriesBar(student_melted, 'Management occupations', "Students")

alt.Chart(...)